In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC

In [3]:
X,Y = make_classification(n_samples = 10000,n_features=10,n_informative=3)


In [4]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

In [5]:
X_train.shape,Y_train.shape

((8000, 10), (8000,))

In [6]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train,Y_train)
y_pred = dt.predict(X_test)
print(accuracy_score(Y_test,y_pred))

0.9175


# Bagging

In [7]:
bag = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators =500,
    max_samples=0.25,
    bootstrap=True,
    random_state=42
)

In [8]:
bag.fit(X_train,Y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_samples=0.25,
                  n_estimators=500, random_state=42)

In [9]:
y_pred1 = bag.predict(X_test)
accuracy_score(Y_test,y_pred1)

0.9445

In [10]:
bag.estimators_samples_

[array([2523, 3113, 7114, ..., 4291, 4472, 3620]),
 array([4782,  663, 7155, ..., 5963,  495, 1767]),
 array([5462, 6574, 4896, ..., 3979, 7827,   37]),
 array([2848, 2629, 1591, ..., 7723, 1314, 1565]),
 array([3821, 6494, 1606, ..., 5686, 7870, 2558]),
 array([2261, 7922, 3649, ..., 4478, 6286, 6943]),
 array([ 652, 1676, 2291, ..., 2723, 7007, 6344]),
 array([2478, 4107, 1958, ..., 7979, 5695, 7854]),
 array([5800, 3548, 6540, ..., 3899,  831,   55]),
 array([5256, 7181, 3409, ..., 5286, 7535, 1335]),
 array([2675, 2834, 3817, ..., 1726, 2323, 7642]),
 array([3236, 7607, 4600, ...,  445, 7501, 6604]),
 array([4563, 4137, 6298, ..., 6611, 3023, 5529]),
 array([2816, 5343, 5817, ..., 3197, 2917, 5775]),
 array([2448, 2733, 5480, ...,  747, 5842,   69]),
 array([4248, 3828, 4630, ..., 1284, 2542, 3370]),
 array([4815, 1867,  503, ..., 4692, 7952, 4855]),
 array([1688, 4132, 5218, ..., 2491, 1876,  471]),
 array([4167, 2900, 3602, ..., 2974, 5705, 1466]),
 array([2920, 7645, 5452, ..., 

# Using SVM

In [11]:
bag1= BaggingClassifier(
    base_estimator = SVC(),
    n_estimators= 300,
    max_samples = 0.25,
    bootstrap = True,
    random_state=42
)

In [12]:
bag1.fit(X_train,Y_train)

BaggingClassifier(base_estimator=SVC(), max_samples=0.25, n_estimators=300,
                  random_state=42)

In [13]:
y_pred2 = bag1.predict(X_test)
accuracy_score(Y_test,y_pred2)

0.932